In [2]:
import requests
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np

In [10]:
nfl = "americanfootball_nfl"
nba = "basketball_nba"
mlb = "baseball_mlb"
ncaaf = "americanfootball_ncaaf"
afl = "aussierules_afl"
nhl = "icehockey_nhl"
nrl = "rugbyleague_nrl"
mls = "soccer_usa_mls"
soccer_brazil = "soccer_brazil_campeonato"
soccer_australia = "soccer_australia_aleague"
soccer_finland = "soccer_finland_veikkausliiga"
soccer_japan = "soccer_japan_j_league"
soccer_ireland = "soccer_league_of_ireland"
soccer_norway = "soccer_norway_eliteserien"
soccer_spain2 = "soccer_spain_segunda_division"
soccer_sweden = "soccer_sweden_allsvenskan"
ucl = "soccer_uefa_european_championship"

In [15]:
# FUNCTIONS

# Define a function to extract the data from the "sites" column and create a new DataFrame
def extract_sites(row):
    home_team = row["teams"][0]
    away_team = row["teams"][1]
    
    site = row["sites"]
    if not site:
        return None
    # Normalize the data in the "sites" column
    sites_df = json_normalize(site)
    sites_df['last_update'] = pd.to_datetime(sites_df['last_update'], unit='s', origin='1969-12-31')
    size = len(sites_df["last_update"])
    sites_df["home_team"] = [home_team] * size
    sites_df["away_team"] = [away_team] * size
    
    def decimal_odds(odds):
        home, away = odds[0], odds[-1]
        
        if home < 0:
            home = 100 / abs(home)
        else:
            home = home / 100
        
        if away < 0:
            away = 100 / abs(away)
        else:
            away = away / 100
        return [round(home, 4), round(away, 4)]
    
    sites_df['odds'] = sites_df['odds.h2h'].apply(decimal_odds)
    sites_df["sportsbook"] = sites_df['site_nice']
    sites_df["home_odds"] = sites_df["odds"].apply(lambda x: x[0])
    sites_df["away_odds"] = sites_df["odds"].apply(lambda x: x[1])
    sites_df = sites_df[['home_team', 'away_team', 'sportsbook','home_odds', 'away_odds']]
    # Return the normalized DataFrame
    return sites_df

def find_arbitrage(df):
    max_home = df.loc[df["home_odds"].idxmax()]
    max_away = df.loc[df["away_odds"].idxmax()]
    
    return guaranteed_profit(max_home["home_odds"], max_away["away_odds"], max_home["sportsbook"], max_away["sportsbook"], max_home["home_team"], max_away["away_team"])

def guaranteed_profit(home, away, home_sportsbook, away_sportsbook, home_team, away_team):
    # Calculate the amount to bet on home to win $100
    x = round(100 / (home + 1), 2)
    # Calculate the amount to bet on away to win $100
    y = round(100 / (away + 1), 2)
    # Calculate the total amount bet
    total_bet = x + y
    # Calculate the profit
    profit = 100 - total_bet
    # If the total amount bet is more than $100, return None
    if total_bet >= 99.9:
        return None
    # Otherwise, return the amount to bet on each side and the profit
    if home >= 1:
        home = str(round(home * 100))
    elif home < 1:
        home = str(round(-100 / home))
        
    if away >= 1:
        away = str(round(away * 100))
    elif away < 1:
        away = str(round(-100 / away))
        
    return pd.DataFrame(
        {"home team": home_team,
         "away team": away_team,
         "home sportsbook": [home_sportsbook],
         "away sportsbook": [away_sportsbook],
         "home bet amount": [x],
         "away bet amount": [y],
         "home team odds": [home],
         "away team odds": [away],
         "roi": [profit]
        }
    )

In [16]:
# Set the API endpoint URL
api_url = "https://api.the-odds-api.com/v3/odds"

# Set the API request parameters
params = {
    "api_key": "f922071b55b41ce5c36770c7ba68ef4a",  # Your API key
    "sport": mls,  # The sport to get odds for (basketball_nba for NBA)
    "region": "us",  # The region to get odds for (us for United States)
    "mkt": "h2h",  # The market to get odds for (h2h for head-to-head)
    "oddsFormat": "american"
}

# Send the API request and get the response
response = requests.get(api_url, params=params)

# Parse the response as JSON
data = response.json()

# Print the sportsbook data
data = pd.DataFrame(data["data"])

In [17]:
ret = pd.DataFrame(columns = ["home team","away team","home sportsbook","away sportsbook","home bet amount","away bet amount","home team odds","away team odds", "roi"])
for _, row in data.iterrows():
    ret = ret.append(find_arbitrage(extract_sites(row)), sort=False)

ret = ret.T
ret.index = ret.index.str.title()
display(ret)

/Users/varunkadikar/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if sys.path[0] == "":


,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Home Team,Atlanta United FC,Minnesota United FC,Charlotte FC,Inter Miami CF,Nashville SC,Colorado Rapids,Chicago Fire,Houston Dynamo,Austin FC,New York City FC,FC Cincinnati,Philadelphia Union,LA Galaxy,FC Dallas
Away Team,D.C. United,Montreal Impact,Seattle Sounders FC,New England Revolution,Toronto FC,Orlando City SC,Columbus Crew SC,Los Angeles FC,Sporting Kansas City,Real Salt Lake,Vancouver Whitecaps FC,San Jose Earthquakes,St. Louis City SC,Portland Timbers
Home Sportsbook,MyBookie.ag,LowVig.ag,LowVig.ag,WynnBET,LowVig.ag,LowVig.ag,WynnBET,LowVig.ag,BetOnline.ag,LowVig.ag,MyBookie.ag,WynnBET,LowVig.ag,WynnBET
Away Sportsbook,BetOnline.ag,Bovada,WynnBET,WynnBET,WynnBET,LowVig.ag,LowVig.ag,LowVig.ag,BetOnline.ag,WynnBET,WynnBET,BetUS,LowVig.ag,WynnBET
Home Bet Amount,52.38,32.47,39.22,14.29,35.34,20.62,40.0,31.75,23.26,27.4,26.67,38.46,27.78,27.78
Away Bet Amount,25.77,28.57,29.41,20.0,29.41,25.51,27.4,27.25,24.69,28.57,26.67,28.17,25.64,27.78
Home Team Odds,-110,208,155,600,183,385,150,215,330,265,275,160,260,260
Away Team Odds,288,250,240,400,240,292,265,267,305,250,275,255,290,260
Roi,21.85,38.96,31.37,65.71,35.25,53.87,32.6,41.0,52.05,44.03,46.66,33.37,46.58,44.44
